<a href="https://colab.research.google.com/github/KaichengDING/Triple-Defense/blob/main/idl_project_creating_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

import matplotlib.pyplot as plt
import time
import logging
import datetime

cuda = torch.cuda.is_available()
cuda

True

In [3]:
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0, dropout=0.5):
        self.std = std
        self.mean = mean
        self.dropout = dropout
        
    def __call__(self, tensor):
      mask = torch.rand(tensor.size()) >= self.dropout
      noise = torch.randn(tensor.size()) * self.std + self.mean
      noise = noise * mask
      res = tensor + noise
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [4]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader): 
            data = data.to(device)
            target = target.to(device)

            with torch.cuda.amp.autocast():
              outputs = model(data)

              _, predicted = torch.max(outputs.data, 1)
              total_predictions += target.size(0)
              correct_predictions += (predicted == target).sum().item()

              loss = criterion(outputs, target).detach()
              running_loss += loss.item()

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [5]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('/content/gdrive/My Drive/IDL_Project/logs/log_{}.log.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [6]:
class ShuffleNet(nn.Module):
    def __init__(self, nb_classes =10):
        super(ShuffleNet, self).__init__()
        self.shuffle = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shuffle(x)
        x = self.linear(x)
        return x

In [9]:
train_batchsize = 128
test_batchsize = 100
num_workers = 8
num_classes = 10

n_epochs = 60
img_size = 224
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4
num_models = 30

noise_std = 0.1
noise_dropout = 0.0

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 
                'min_lr': min_lr, 
                'weight_decay': weight_decay, 
                'num_models': num_models, 
                'num_epochs': n_epochs, 
                'noise_std': noise_std, 
                'noise_dropout':noise_dropout}
logging.info(hyper_params)


2020-12-06 05:31:38,774 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 30, 'num_epochs': 60, 'noise_std': 0.1, 'noise_dropout': 0.0}


In [10]:
seeds = np.arange(0, num_models)

for seed in seeds:
  logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)
  

  train_transform = transforms.Compose([transforms.Resize(size=img_size),
                      transforms.ColorJitter(hue=.05, saturation=.05),
                      transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                      transforms.ToTensor(),
                      transforms.Normalize((0, 0, 0), (1, 1, 1)),
                      AddGaussianNoise(0., noise_std, noise_dropout)])

  test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1)),
                     AddGaussianNoise(0., noise_std, noise_dropout)])

  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  model = ShuffleNet(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  # Train_loss = []
  # Test_loss = []
  # Test_acc = []

  for i in range(n_epochs):
    tic = time.time()
    logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    # Train_loss.append(train_loss)
    # Test_loss.append(test_loss)
    # Test_acc.append(test_acc)
    toc = time.time()
    logging.info('Time: {}s'.format(toc - tic))
    logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/content/gdrive/My Drive/IDL_Project/modelS/Model_shufflenet{}_std{}_dropout{}".format(seed, noise_std, noise_dropout))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

  


2020-12-06 05:31:41,067 INFO Seed: 0


Files already downloaded and verified
Files already downloaded and verified


2020-12-06 05:31:42,961 INFO Epoch: 0
2020-12-06 05:36:03,765 INFO Training Loss: 1.9401713414570254
2020-12-06 05:36:03,767 INFO Training Accuracy: 28.198%
2020-12-06 05:36:28,050 INFO Testing Loss: 1.7628186535835266
2020-12-06 05:36:28,052 INFO Testing Accuracy: 35.75%
2020-12-06 05:36:28,053 INFO Time: 285.09193444252014s
2020-12-06 05:36:28,054 INFO ====================
2020-12-06 05:36:28,056 INFO Epoch: 1
2020-12-06 05:40:50,859 INFO Training Loss: 1.6809083250782373
2020-12-06 05:40:50,862 INFO Training Accuracy: 37.976%
2020-12-06 05:41:15,523 INFO Testing Loss: 1.5854601502418517
2020-12-06 05:41:15,525 INFO Testing Accuracy: 42.199999999999996%
2020-12-06 05:41:15,526 INFO Time: 287.4700357913971s
2020-12-06 05:41:15,527 INFO ====================
2020-12-06 05:41:15,529 INFO Epoch: 2
2020-12-06 05:45:39,776 INFO Training Loss: 1.5419034360314878
2020-12-06 05:45:39,778 INFO Training Accuracy: 43.146%
2020-12-06 05:46:04,502 INFO Testing Loss: 1.4594215989112853
2020-12-06 05

Epoch    25: reducing learning rate of group 0 to 1.0000e-05.


2020-12-06 07:37:06,119 INFO Training Loss: 0.2995814990890605
2020-12-06 07:37:06,121 INFO Training Accuracy: 89.778%
2020-12-06 07:37:30,826 INFO Testing Loss: 0.5663109876215457
2020-12-06 07:37:30,827 INFO Testing Accuracy: 81.38%
2020-12-06 07:37:30,829 INFO Time: 291.3675925731659s
2020-12-06 07:37:30,831 INFO ====================
2020-12-06 07:37:30,832 INFO Epoch: 26
2020-12-06 07:41:57,094 INFO Training Loss: 0.26584649194613136
2020-12-06 07:41:57,096 INFO Training Accuracy: 90.996%
2020-12-06 07:42:22,111 INFO Testing Loss: 0.5604530218243599
2020-12-06 07:42:22,113 INFO Testing Accuracy: 81.42%
2020-12-06 07:42:22,114 INFO Time: 291.2822616100311s
2020-12-06 07:42:22,115 INFO ====================
2020-12-06 07:42:22,117 INFO Epoch: 27
2020-12-06 07:46:49,153 INFO Training Loss: 0.2529543177474795
2020-12-06 07:46:49,155 INFO Training Accuracy: 91.418%
2020-12-06 07:47:14,100 INFO Testing Loss: 0.569378237426281
2020-12-06 07:47:14,102 INFO Testing Accuracy: 81.74%
2020-12-0

...

Files already downloaded and verified
Files already downloaded and verified


2020-12-06 15:14:32,252 INFO Epoch: 0
2020-12-06 15:18:51,581 INFO Training Loss: 1.9603939833848372
2020-12-06 15:18:51,583 INFO Training Accuracy: 27.107999999999997%
2020-12-06 15:19:16,187 INFO Testing Loss: 1.8195032858848572
2020-12-06 15:19:16,189 INFO Testing Accuracy: 31.900000000000002%
2020-12-06 15:19:16,190 INFO Time: 283.9382474422455s
2020-12-06 15:19:16,192 INFO ====================
2020-12-06 15:19:16,193 INFO Epoch: 1
2020-12-06 15:23:33,780 INFO Training Loss: 1.6921703169108047
2020-12-06 15:23:33,781 INFO Training Accuracy: 37.702000000000005%
2020-12-06 15:23:58,099 INFO Testing Loss: 1.5884682512283326
2020-12-06 15:23:58,101 INFO Testing Accuracy: 40.01%
2020-12-06 15:23:58,102 INFO Time: 281.90831303596497s
2020-12-06 15:23:58,104 INFO ====================
2020-12-06 15:23:58,105 INFO Epoch: 2
2020-12-06 15:28:14,960 INFO Training Loss: 1.5596873138261877
2020-12-06 15:28:14,962 INFO Training Accuracy: 42.622%
2020-12-06 15:28:39,743 INFO Testing Loss: 1.515371

Epoch    26: reducing learning rate of group 0 to 1.0000e-05.


2020-12-06 17:20:32,571 INFO Training Loss: 0.2611273646240344
2020-12-06 17:20:32,572 INFO Training Accuracy: 91.096%
2020-12-06 17:20:56,319 INFO Testing Loss: 0.5651213747262954
2020-12-06 17:20:56,321 INFO Testing Accuracy: 81.73%
2020-12-06 17:20:56,322 INFO Time: 273.25961446762085s
2020-12-06 17:20:56,322 INFO ====================
2020-12-06 17:20:56,323 INFO Epoch: 27
2020-12-06 17:25:05,291 INFO Training Loss: 0.2337536294861218
2020-12-06 17:25:05,293 INFO Training Accuracy: 92.238%
2020-12-06 17:25:28,904 INFO Testing Loss: 0.5656902304291725
2020-12-06 17:25:28,906 INFO Testing Accuracy: 82.16%
2020-12-06 17:25:28,908 INFO Time: 272.58446764945984s
2020-12-06 17:25:28,909 INFO ====================
2020-12-06 17:25:28,911 INFO Epoch: 28
2020-12-06 17:29:38,311 INFO Training Loss: 0.22340632477760924
2020-12-06 17:29:38,313 INFO Training Accuracy: 92.522%
2020-12-06 17:30:01,767 INFO Testing Loss: 0.5686032471060752
2020-12-06 17:30:01,769 INFO Testing Accuracy: 82.07%
2020-1

KeyboardInterrupt: ignored